<a href="https://colab.research.google.com/github/yukinaga/object_detection/blob/main/section_2/01_faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faster R-CNNによる物体検出
PyTorchを使って、Faster R-CNNによる物体検出を実装します。

## 設定
必要なライブラリの導入します。  
また、インデックスを物体名に変換するためのリスト、および物体名をインデックスに変換するための辞書を用意しておきます。

In [ ]:
import torch
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import draw_bounding_boxes
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import numpy as np
import matplotlib.pyplot as plt

# インデックスを物体名に変換
index2name = [
    "person",
    "bird",
    "cat",
    "cow",
    "dog",
    "horse",
    "sheep",
    "aeroplane",
    "bicycle",
    "boat",
    "bus",
    "car",
    "motorbike",
    "train",
    "bottle",
    "chair",
    "diningtable",
    "pottedplant",
    "sofa",
    "tvmonitor",
]
print(index2name)

# 物体名をインデックスに変換
name2index = {}
for i in range(len(index2name)):
    name2index[index2name[i]] = i
print(name2index)

## ターゲットを整える関数
バウンディングボックスおよび物体名のデータはxml形式で格納されています。  
ここから必要なデータを抽出し、整えるための関数を用意します。  

In [ ]:
def arrange_target(target):
    objects = target["annotation"]["object"]
    box_dics = [obj["bndbox"] for obj in objects]
    box_keys = ["xmin", "ymin", "xmax", "ymax"]

    # バウンディングボックス
    boxes = []
    for box_dic in box_dics:
        box = [int(box_dic[key]) for key in box_keys]
        boxes.append(box)
    boxes = torch.tensor(boxes)

    # 物体名
    labels = [name2index[obj["name"]] for obj in objects]  # 物体名はインデックスに変換
    labels = torch.tensor(labels)

    dic = {"boxes":boxes, "labels":labels}
    return dic

## データセットの読み込み
Torchvisionが用意しているデータセット、「Pascal VOC Detection Dataset」を読み込みます。  
https://pytorch.org/vision/0.8/datasets.html#torchvision.datasets.VOCDetection  
`transform`、および`target_transform`の設定を行い、使用する際にデータを整えます。

In [ ]:
dataset_train=torchvision.datasets.VOCDetection(root="./VOCDetection/2012",
                                                year="2012",image_set="train",
                                                download=True,
                                                transform=transforms.ToTensor(),
                                                target_transform=transforms.Lambda(arrange_target)
                                                )

dataset_test=torchvision.datasets.VOCDetection(root="./VOCDetection/2012",
                                                year="2012",image_set="val",
                                                download=True,
                                                transform=transforms.ToTensor(),
                                                target_transform=transforms.Lambda(arrange_target)
                                                )

## DataLoaderの設定
DataLoaderを設定し、データを少しずつ取り出せるようにします。  
今回はtargetのデータ形状が毎回異なるので、バッチサイズは1に設定します。  

In [ ]:
data_loader_train =  DataLoader(dataset_train, batch_size=1, shuffle=True)
data_loader_test =  DataLoader(dataset_train, batch_size=1, shuffle=True)

## ターゲットの表示
画像上にバウンディングボックスとラベルを描画します。  
これらの描画には、`draw_bounding_boxes`関数を使用します。  
https://pytorch.org/vision/master/utils.html#torchvision.utils.draw_bounding_boxes  

In [ ]:
def show_boxes(image, boxes, names):
    drawn_boxes = draw_bounding_boxes(image, boxes, labels=names)

    plt.figure(figsize = (16,16))
    plt.imshow(np.transpose(drawn_boxes, (1, 2, 0)))  # チャンネルを一番後ろに
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # ラベルとメモリを非表示に
    plt.show()

dataiter = iter(data_loader_train)  # イテレータ
image, target = dataiter.next()  # バッチを取り出す
print(target)

image = image[0]
image = (image*255).to(torch.uint8)  # draw_bounding_boxes関数の入力は0-255

boxes = target["boxes"][0]

labels = target["labels"][0]
names = [index2name[label.item()] for label in labels]

show_boxes(image, boxes, names)

# モデルの構築
ResNet-50-FPNをバックボーンに持つFaster R-CNNモデルを設定し、学習済みのパラメータを読み込みます。  
https://pytorch.org/vision/stable/models.html#torchvision.models.detection.fasterrcnn_resnet50_fpn

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes=len(index2name)+1  # 分類するクラス数
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.cuda()  # GPU対応

## 訓練
用意したデータ、構築したモデルを使って学習を行います。

In [ ]:
# 最適化アルゴリズム
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

model.train()  # 訓練モード
epochs = 1
for epoch in range(epochs):
    for i, (image, target) in enumerate(data_loader_train):
        image = [img.cuda() for img in image]  # GPU対応

        boxes = target["boxes"][0].cuda()
        labels = target["labels"][0].cuda()
        target = [{"boxes":boxes, "labels":labels}]  # ターゲットは辞書を要素に持つリスト

        loss_dic = model(image, target)
        loss = sum(loss for loss in loss_dic.values())  # 誤差の合計を計算

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i%100 == 0:  # 100エポックごとに経過を表示
            print("epoch:", epoch,  "iteration:", i,  "loss:", loss.item()) 

## 訓練したモデルの使用
訓練済みのモデルを使って予測を行います。  
この時点では対象のスコアを考慮してないので、多数のバウンディングボックスとラベルが表示されます。

In [ ]:
dataiter = iter(data_loader_test)  # イテレータ
image, target = dataiter.next()  # バッチを取り出す

image = [img.cuda() for img in image]  # GPU対応

model.eval()
predictions = model(image)
print(predictions)

image = (image[0]*255).to(torch.uint8).cpu() # draw_bounding_boxes関数の入力は0-255
boxes = predictions[0]["boxes"].cpu()
labels = predictions[0]["labels"].cpu().detach().numpy()
labels = np.where(labels>=len(index2name), 0, labels)  # ラベルが範囲外の場合は0に
names = [index2name[label.item()] for label in labels]

print(names)
show_boxes(image, boxes, names)

## スコアによる選別
バウンディングボックスとラベルをスコアにより選別します。

In [ ]:
boxes = []
names = []
for i, box in enumerate(predictions[0]["boxes"]):
    score = predictions[0]["scores"][i].cpu().detach().numpy()
    if score > 0.5:  # スコアが0.5より大きいものを抜き出す
        boxes.append(box.cpu().tolist())
        label = predictions[0]["labels"][i].item()
        if label >= len(index2name):  # ラベルが範囲外の場合は0に
            label = 0
        name = index2name[label]
        names.append(name)
boxes = torch.tensor(boxes)

show_boxes(image, boxes, names)